In [10]:
import numpy as np
from scipy.io import loadmat
from sklearn.cluster import KMeans
from skmultilearn.adapt import MLkNN
from sklearn.metrics import f1_score
from sklearn.metrics import label_ranking_loss
from sklearn.metrics import hamming_loss
from sklearn.metrics import average_precision_score
from sklearn.metrics import coverage_error
from sklearn.metrics import zero_one_loss
from sklearn.neighbors import kneighbors_graph
import sklearn
import matplotlib.pyplot as plt
import pandas as pd
import sklearn.metrics as metrics
import warnings
warnings.filterwarnings('ignore')


In [ ]:
data = loadmat(f'../../../Datasets/Arts.mat')
train = data['train']
test = data['test']

X_test = train[0][0].T
Y_test = train[0][1].T
Y_test[Y_test == -1] = 0

X =  test[0][0].T
Y =  test[0][1].T
Y[Y == -1] = 0

XTX = X.T @ X

#Feature
n,d = X.shape
#label
n,c = Y.shape

l1  =  0.01
l2  =  0.01
l3  =  0.01
l4  =  0.01

sigma = 1
nn = 5
S= kneighbors_graph(Y.T, nn, mode='distance', include_self=False).toarray()
#     S  = np.exp(-dis/(sigma))
A = np.diag(np.sum(S, axis=1))

eps = np.finfo(np.float16).eps
ppn=0

W = np.random.rand(d,c)
Z = np.random.rand(c,c)

t = 100

for i in range(t):
    
    Qn = np.linalg.norm(2 * W,ord=1)
    Q  = 1 / np.maximum(Qn,eps)

    Z21 = np.linalg.norm(Z, ord=2, axis=1)
    Z_21 = 1 / np.maximum(Z21,eps)
    DZ_21 = np.diag(Z_21)

    Wu = X.T @ Y @ Z
    Wd = X.T @ X @ W + l4 * (Q * W)  
    W  = W * (Wu / np.maximum(Wd,eps))

    Zu = Y.T @ X @ W + l1 * (Y.T @ Y) + l2 * (Y.T @ Y @ Z @ S) 
    Zd = Y.T @ Y @ Z + l1 * (Y.T @ Y @ Z @ A) + l3 * (DZ_21 @ Z) 
    Z  = Z * (Zu / np.maximum(Zd,eps))

WW = np.linalg.norm(W,axis=1,ord=2)
sQ = np.argsort(WW)
nosf = int (20 * d / 100)
sX = X[:,sQ[d-nosf:]]
classifier = MLkNN(k=10)
classifier.fit(sX, Y)
# predict
predictions = classifier.predict(X_test[:,sQ[d-nosf:]]).toarray()
scores = classifier.predict_proba(X_test[:,sQ[d-nosf:]]).toarray()
MIC = f1_score(Y_test, predictions, average='micro')
MAC = f1_score(Y_test, predictions, average='macro')
AVP = average_precision_score(Y_test.T,scores.T)
HML = hamming_loss(Y_test,predictions)
RNL = label_ranking_loss(Y_test,scores)
ZER = zero_one_loss(Y_test,predictions)
CVE = coverage_error(Y_test,scores)

print(  'Micro-F1 :',MIC,'\n',
        'Macro-F1 :',MAC,'\n',
        'Average Precision :',AVP,'\n',
        'Hamming Loss :',HML,'\n',
        'Ranking Loss :',RNL,'\n',
        'Zero-One Loss :',ZER,'\n',
        'Coverage Error:',CVE)
